# Data Acquisition Example: Testing WLAN Client Receiver System Noise
_By Dan Kuester_

Test data for WLAN is implemented with the the iperf support in [ssmdevices](https://gitlab.nist.gov/ssm/ssmdevices). It produces many more columns of data than we need, so we just use the two defined in `ipc_columns`: throughput and a timestamp.

The results of the tests and the corresponding test conditions are stored in a [flat database](https://en.wikipedia.org/wiki/Flat_file_database) stored in SQLite format. It is implemented as a typical intended use case of `labbench.RelationalDataInSQLite`. The test conditions are implemented as states in the attenuators, `iperf`, and `wlan`, so all we need to log these results into the database is to add the `db.on_set` call. Any states that are changed after that function call (for example, with the for loops in the acquisition code) become columns in the database; these values are automatically kept up to date and written to the database on calls to `db.write`.

In [ ]:
# -*- coding: utf-8 -*-
# @author: ynm5; revisions by dkuester

%pylab inline
import time
import ssmdevices as ssm
import labbench as lb
import pandas as pd

def meshpoints (x1, x2, *args):
    ''' For input of N-dimensions of grid point vectors (x1, x2, ..., xN),
        where each (x1, x2, ... xN) has dimension (M1, M2, ..., MN),
        returns an array with shape (M1*M2*...*MN,N) that consists of the
        unique points in the multi-dimensional grid with sampled at
        the vector points (x1, x2, ..., xN) in each axis. This is also known
        as the the cartesian product of (x1 ... xN).
    '''
    ret = np.array(np.meshgrid(*((x1,x2)+args)))
    return ret.reshape([ret.shape[0],np.prod(ret.shape[1:])]).T

setup_time = 20 # seconds
acquisition_time = 20 seconds

iperf_columns       = ['iperf_bits_per_second','iperf_timestamp']
sweep_points       = meshpoints(range(20,61,20),[110]+range(0,41,40))
#np.random.shuffle(c_e0_points) # randomize?

db_path = '{}.db'.format(time.strftime("%Y-%m-%d-%HH-%MM"))

lb.debug_to_screen(level=lb.DEBUG)

## This `with' block ensures sockets and iperf subprocesses are
## closed correctly when the script ends, even if exceptions are raised
with ssm.instruments.MiniCircuitsRCDAT('11604210023')     as c,\
     ssm.instruments.MiniCircuitsRCDAT('11604210014')     as e0,\
     ssm.software.IPerfClient('10.0.0.3', interval=0.1)   as iperf,\
     ssm.software.WLANStatus('Wi-Fi', ssid='Engenius1')   as wlan,\
     lb.StatesToSQLite(db_path, 'results')                as db:
     
    # Log all state changes in lte_laa and each client in iperf_clients
    db.observe([c,e0,iperf,wlan])

    # Only need to do this if there is a variable attenuator in the test setup
    # lte_enb_atten.state.attenuation = 20. # dB

    print 'Pausing for {}s setup time'.format(setup_time)
    c.state.attenuation = 30
    e0.state.attenuation = 110
    time.sleep(setup_time)
    df=iperf.fetch()
    
    row = {}
    i = 0
    for c.state.attenuation, e0.state.attenuation in sweep_points:
        i += 1
        print '{}/{}: atten C {} dB, atten E0 {} dB'\
              .format(i, len(sweep_points), c.state.attenuation, e0.state.attenuation)
            
        time.sleep(acquisition_time)
        
        for sample,row in iperf.fetch().iterrows():
            db.append(**row.to_dict())
        db.write()

# Load the sqlite database and save a copy into a csv
df = lb.read_sqlite(db_path, 'results')
df.to_csv(db_path+'.csv')

Populating the interactive namespace from numpy and matplotlib


2017-09-05 14:28:26 - labbench - DEBUG - starting WLAN reconnect watchdog


Pausing for 5s setup time
1/9: atten C 20.0 dB, atten E0 110.0 dB
2/9: atten C 40.0 dB, atten E0 110.0 dB
3/9: atten C 60.0 dB, atten E0 110.0 dB
4/9: atten C 20.0 dB, atten E0 0.0 dB
5/9: atten C 40.0 dB, atten E0 0.0 dB
6/9: atten C 60.0 dB, atten E0 0.0 dB
7/9: atten C 20.0 dB, atten E0 40.0 dB
8/9: atten C 40.0 dB, atten E0 40.0 dB
9/9: atten C 60.0 dB, atten E0 40.0 dB


In [4]:
df

,c_attenuation,e0_attenuation,iperf_bits_per_second,iperf_destination_address,iperf_destination_port,iperf_source_address,iperf_source_port,iperf_timestamp,time
id,,,,,,,,,
0,20.0,110.0,970981376,10.0.0.3,5001,10.0.0.3,49196,2017-09-05 14:28:31.000,2017-09-05 14:28:34.325
1,20.0,110.0,789381120,10.0.0.3,5001,10.0.0.3,49196,2017-09-05 14:28:31.250,2017-09-05 14:28:34.371
2,20.0,110.0,981204992,10.0.0.3,5001,10.0.0.3,49196,2017-09-05 14:28:31.500,2017-09-05 14:28:34.387
3,20.0,110.0,977698816,10.0.0.3,5001,10.0.0.3,49196,2017-09-05 14:28:32.750,2017-09-05 14:28:34.418
4,20.0,110.0,1022623744,10.0.0.3,5001,10.0.0.3,49196,2017-09-05 14:28:32.000,2017-09-05 14:28:34.450
5,20.0,110.0,991166464,10.0.0.3,5001,10.0.0.3,49196,2017-09-05 14:28:32.250,2017-09-05 14:28:34.465
6,20.0,110.0,971735040,10.0.0.3,5001,10.0.0.3,49196,2017-09-05 14:28:32.500,2017-09-05 14:28:34.496
7,20.0,110.0,1034485760,10.0.0.3,5001,10.0.0.3,49196,2017-09-05 14:28:33.750,2017-09-05 14:28:34.528
8,20.0,110.0,744685568,10.0.0.3,5001,10.0.0.3,49196,2017-09-05 14:28:33.000,2017-09-05 14:28:34.543
